In [1]:
import sys
sys.path.append("../src")
from utils import *
import warnings
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)

from sklearn.impute import KNNImputer

import pandas as pd
import numpy as np

### Load

In [2]:
#Load accident datafrma
df_acc = load_accident()
df_acc.dropna(subset=['lat', 'long'], axis=0, inplace=True)

In [3]:
df_acc.head()

,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,lum,int,atm,col,com,adr,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg
0,1,0.0,3.0,0.0,2.0,0.0,9,2.0,NaN,3.0,1.0,1.0,1.0,0.0,1.0,1,0,1.0,3.0,12.0,"., FRANCE ( AVE ANATOLE",47.74613,-3.36700,560,0.0,2.0,0.0,2010-12-11,1
1,2,0.0,3.0,0.0,2.0,3.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,1.0,94.0,rue de tanouarn,48.35744,-1.71484,350,NaN,NaN,NaN,2011-01-19,1
2,2,0.0,3.0,0.0,2.0,1.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,3.0,58.0,rue v.hugo/rue debussy,43.86300,1.49563,310,0.0,2.0,3.0,2011-04-20,1
3,1,0.0,3.0,0.0,9.0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1,5,1.0,6.0,73.0,NaN,49.65811,3.31329,20,0.0,2.0,0.0,2011-04-21,1
4,2,0.0,3.0,0.0,0.0,1.0,4,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1,2,1.0,1.0,51.0,rue de l'Erbonière,48.10537,-1.62901,350,0.0,9.0,3.0,2011-05-13,1


In [5]:
#Load infra Dataframe
df_infra = load_infra()

### Link

In [48]:
#def linking function
import numpy as np
from math import sin, cos, sqrt, atan2, radians, asin
from tqdm import tqdm
import warnings

warnings.simplefilter('ignore')

def distance(lat_acc, lon_acc, lat_fin, lon_fin, lat_dep, lon_dep):
    """
    Compute distances between accident point and infratructures point
    """
    R = 6371
    lat_dep, lon_dep, lat_acc, lon_acc = map(np.radians, [lat_dep, lon_dep, lat_acc, lon_acc])
    
    dlat = lat_acc - lat_dep
    dlon = lon_acc - lon_dep
    
    a = np.sin(dlat / 2)**2 + np.cos(lat_dep) * np.cos(lat_acc) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance1 = R * c
    
    lat_fin, lon_fin = map(np.radians, [lat_fin, lon_fin])
    
    dlat = lat_acc - lat_fin
    dlon = lon_acc - lon_fin
    
    a = np.sin(dlat / 2)**2 + np.cos(lat_fin) * np.cos(lat_acc) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance2 = R * c

    if(distance1.min()>distance2.min()):
        return distance2
    else:
        return distance1

def link_dataframes(accidents, infrastructure, radius):
    """
    Add index of infrastucture to the accident dataframe (and keep distances of this infra)
    """
    # On ajoute une colonne distance qui sera utilisé pour la jointure

    infrastructure['index'] = infrastructure.index
    accidents['distance'] = np.nan
    accidents['index_in_infra'] = np.nan
    for i, row in tqdm(accidents.iterrows(), total=accidents.shape[0]):
        lat1, lon1 = row['lat'], row['long']
        
        infrastructure['distance'] = distance(lat1, lon1,
                                              infrastructure['latitude_fin'],
                                              infrastructure['longitude_fin'],
                                              infrastructure['latitude_dep'],
                                              infrastructure['longitude_dep'])
        distance_series = infrastructure.query("distance <= @radius").distance
        if distance_series.size == 0:
            accidents.at[i, 'distance'] = -1
            accidents.at[i, 'index_in_infra'] = -1
        else:
            accidents.at[i, 'distance'] = distance_series.min()*1000
            accidents.at[i, 'index_in_infra'] = distance_series.argmin()
    
    infrastructure.drop(["distance"], inplace=True, axis=1)
    #accidents.drop(["index"], inplace=True, axis=1)
    
    return accidents, infrastructure

### Merge

In [60]:
df_acc_withIndex, df_infra = link_dataframes(df_acc[-600:], df_infra, 500) #link

100%|██████████| 600/600 [00:43<00:00, 13.83it/s]


In [58]:
##MERGE
df_acc_withIndex['index_in_infra'] = df_acc_withIndex['index_in_infra'].astype("Int32")

df_merged = df_acc_withIndex.merge(df_infra, left_on="index_in_infra", right_on="index")

### Map

In [55]:
import folium

def make_map(df):
    # Create a map object
    m = folium.Map(location=[48.8, 2.3], zoom_start=12)

    # Create the layer control
#     layer_control = folium.map.LayerControl(position='topright', collapsed=False)

    # Iterate through the rows of your dataset
    for i, row in df.iterrows():
        # Calculate the size of the marker based on the gravity of the accident
        size = row['grav'] * 10
        # Add a marker for the latitude and longitude
        accident = folium.RegularPolygonMarker(location=[row['lat'], row['long']], 
                                    number_of_sides=3, 
                                    radius=size,
                                    color='red', 
                                    fill_color='red')
        accident.add_to(m)
        #layer_control.add_child(accident, name="Accidents (proportional to gravity)")
        # Add a marker for the latitude_dep and longitude_dep
        infra1 = folium.CircleMarker(location=[row['latitude_dep'], row['longitude_dep']], 
                            radius=5, 
                            color='blue', 
                            fill=True, 
                            fill_color='blue')
        infra1.add_to(m)
        #layer_control.add_child(infra, name="Infrastructure (Cycling)")
        infra2 = folium.CircleMarker(location=[row['latitude_fin'], row['longitude_fin']], 
                            radius=5, 
                            color='blue', 
                            fill=True, 
                            fill_color='blue')
        infra2.add_to(m)
        # Draw a line between the latitude_dep, longitude_dep and latitude_fin, longitude_fin
        folium.PolyLine([[row['latitude_dep'], row['longitude_dep']], 
                         [row['latitude_fin'], row['longitude_fin']]], 
                        color='green', 
                        weight=2.5, 
                        opacity=1).add_to(m)

    # Add the layer control to the map
    #layer_control.add_to(m)
    # Show the map
    #m.save("accident_with_infra.html")
    display(m)

In [59]:
make_map(df_merged)

### Experiment

In [89]:
from geopy.distance import great_circle

df_infra_ = df_infra.copy()
df_acc_ = df_acc.copy()

#### Selected infra and accident in top 10 Citys

In [90]:
def filter_top_nine_citys_only(df_acc_, df_infra_):
    top_nine_citys = [75,44,33,59,31,13,69,34,67]
    mask_infra = np.zeros(df_infra_.shape[0]).astype(bool)
    mask_acc = np.zeros(df_acc_.shape[0]).astype(bool)

    for code in top_nine_citys:
        mask_acc = np.logical_or(mask_acc, df_acc_['com'] == code)
        mask_infra = np.logical_or(mask_infra, np.logical_or(df_infra_['code_com_g'] == code, df_infra_['code_com_d'] == code))

    df_infra_ = df_infra_[mask_infra]
    df_acc_ = df_acc_[mask_acc]

    return df_acc_.reset_index(drop=True), df_infra_.reset_index(drop=True)


In [91]:
df_acc_, df_infra_ = filter_top_nine_citys_only(df_acc_, df_infra_)

In [92]:
def closest_point(A, B, C):
    """
    Point le plus proche de A appartenant à la droit BC
    """
    A = np.array(A)
    B = np.array(B)
    C = np.array(C)
    AB = C - B 
    AC = A - B
    dot = np.dot(AB, AC)
    proj = dot / np.dot(AB, AB)
    P = B + proj * AB
    return P

def distance_infra_acc(raw, lat_acc, long_acc):
            point = closest_point([lat_acc,long_acc], [raw['latitude_dep'], raw['longitude_dep']], [raw['latitude_fin'], raw['longitude_fin']])
            dist = great_circle((lat_acc,long_acc), point).meters
            return dist

In [93]:
print(df_acc_.shape)
print(df_infra_.shape)

(6359, 29)
(81201, 32)


In [116]:
def associate_infra_index_to_acc(df_acc, df_infra):

    df_acc_ = df_acc.iloc[:20, :].copy()
    df_infra_ = df_infra.copy()

    top_nine_citys = [75,44,33,59,31,13,69,34,67]

    df_acc_["distance_to_infra"] = np.nan
    df_acc_["infra_index"] = np.nan

    for code in top_nine_citys:

        print(code)

        #Select a sub-dataset (a city) in order to reduce research computations among infrastructure
        mask_acc = df_acc_['com'] == code
        mask_infra = np.logical_or(df_infra_['code_com_g'] == code, df_infra_['code_com_d'] == code)
        df_infra_city = df_infra_[mask_infra]
        df_acc_city = df_acc_[mask_acc]
        
        for i_acc in range(df_acc_city.shape[0]):
            acc = df_acc_city.iloc[i_acc,:]
            acc_index = df_acc_city.iloc[i_acc:i_acc+1,:].index.values[0] #subtefuge pour obtenir l'index
            lat_acc = acc['lat']
            long_acc = acc['long']
            
            #calcul la distance d'un accident à tout les infrastructure d'une même ville
            dist_series = df_infra_city[['latitude_dep', 'longitude_dep', 'latitude_fin', 'longitude_fin']].apply(distance_infra_acc, axis=1, args=(lat_acc, long_acc))
            
            #Selectionne l'infra la plus proche
            df_acc_.loc[acc_index, "infra_index"] = dist_series.argmin() #reference index of infrastructure
            df_acc_.loc[acc_index, "distance_to_infra"] = dist_series.min()
            print(dist_series.argmin(), "=> loc (", acc_index)
            print(dist_series.min())
    
    return df_acc_, df_infra_




In [117]:
acc, infra = associate_infra_index_to_acc(df_acc_, df_infra_)

75
1114 => loc ( 9
2.388243914298859
605 => loc ( 11
0.596396488813632
5691 => loc ( 15
0.20127584391325115
44
2826 => loc ( 1
299.3168543921456
8342 => loc ( 5
12.379612594813146
6615 => loc ( 16
0.25060441271410894
613 => loc ( 18
0.23814150529467998
33
7320 => loc ( 2
75.81167813616966
910 => loc ( 14
0.05732247263529472
8678 => loc ( 17
4.8637413004317125
59
31
6457 => loc ( 0
17.675394930564096
13
1372 => loc ( 3
334.29262968788044
2484 => loc ( 10
0.7129153838940915
69
8572 => loc ( 6
0.3508993579550904
34
4299 => loc ( 4
175.80090418438752
1009 => loc ( 7
1.2430641311778863
847 => loc ( 8
1.301324370842688
67
3264 => loc ( 12
0.017658339834586152
34 => loc ( 13
2.0120035773249274
8200 => loc ( 19
0.5202149013461498


In [118]:
acc.head()

,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,lum,int,atm,col,com,adr,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg,distance_to_infra,infra_index
0,2,1.0,NaN,10.0,0.0,1.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1.0,7.0,31.0,D 929,42.89710,3.53210,650,NaN,NaN,NaN,2016-06-28,1,17.675395,6457.0
1,1,2.0,3.0,0.0,2.0,1.0,4,2.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1.0,1.0,44.0,"0, VENDENHEIM (RUE DE)",48.60924,7.74464,670,0.0,2.0,1.0,2016-07-31,1,299.316854,2826.0
2,1,0.0,2.0,0.0,9.0,2.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,9.0,3.0,33.0,rue de l'industrie,46.10551,5.82006,10,0.0,2.0,1.0,2017-03-29,1,75.811678,7320.0
3,1,0.0,3.0,0.0,2.0,1.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1.0,1.0,13.0,rue du calvaire,48.42815,7.65991,670,0.0,2.0,1.0,2018-03-03,1,334.292630,1372.0
4,2,0.0,NaN,12.0,0.0,1.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1.0,6.0,34.0,allée des acacias,49.57179,2.96532,600,NaN,NaN,NaN,2018-06-15,1,175.800904,4299.0


In [103]:
acc["distance_to_infra"].value_counts(dropna=False)

NaN    20
Name: distance_to_infra, dtype: int64

In [109]:
a = pd.DataFrame([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,10,15,16]], columns=['un', 'deux', 'trois', 'quatre'])
a

,un,deux,trois,quatre
0,1,2,3,4
1,5,6,7,8
2,9,10,11,12
3,13,10,15,16


In [114]:
a

,un,deux,trois,quatre
0,1,2,3,4
1,5,6,7,8
2,9,10,11,12
3,13,10,15,16


In [112]:
b = a[a['deux'] == 10]
b

,un,deux,trois,quatre
2,9,10,11,12
3,13,10,15,16


In [115]:
b

,un,deux,trois,quatre
2,9,10,11,12
3,13,10,15,0


In [113]:
b.loc[3, 'quatre'] = 0

In [88]:
b.loc[3]

0    13
1    10
2    15
3    16
Name: 3, dtype: int64

In [86]:
for i in range(b.shape[0]):
    print(b.iloc[i:i+1,:].index.values)

[2]
[3]


In [ ]:
#merged function
for i_acc in range(df_acc_.shape[0]):
    df_acc_.iloc[i_acc,:]['']
